## References and important URLS requirements- 

- [1] https://openai.com/blog/function-calling-and-other-api-updates
- [2] https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
- [3] https://rapidapi.com/weatherapi/api/weatherapi-com/
- [4] https://platform.openai.com/account/api-keys

In [1]:
import os
ROOT = "/workspaces/gpt-function-calling-demo/"
os.chdir(ROOT)
os.getcwd()

'/workspaces/gpt-function-calling-demo'

# Previous ways - 

## Via Openai Python ChatCompletion class- 

```python
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
```

## Via CURL command

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'
```

## Via requests module

```python
import requests

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + OPENAI_API_KEY,
}
json_data = {
    "model": "gpt-3.5-turbo", 
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": "Hello!"}
        ]}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data,
)
```


## Weather Chatbot example

In [2]:
import requests
from dotenv import load_dotenv
import json
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GPT_MODEL = "gpt-3.5-turbo-0613" # new model for function calling abilities

## Checkout the following example - 

```bash
curl https://api.openai.com/v1/chat/completions -u :$OPENAI_API_KEY -H 'Content-Type: application/json' -d '{
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}'
```

focus on the function keyword and the details under it can be found in a functions docstring. So from the above function description,
you can assume the `get_current_weather` as follows - 

```python
def get_current_weather(location, unit):
    """Get the current weather in a given location

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        unit (str): ["celsius", "fahrenheit"]
    """
    pass
```

In [3]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY,
    }
    json_data = {
        "model": model, 
        "messages": messages
        }
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )

        return response
    except Exception as e:
        print(f"\n>>>> Unable to generate ChatCompletion response")
        print(f"\n>>>> Exception: {e}")
        raise e

In [4]:
# simple use

sample_messages = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "user", "content": "Hi there"},
]

chat_response = chat_completion_request(messages=sample_messages)

chat_response

<Response [200]>

In [5]:
chat_response.json()

{'id': 'chatcmpl-7zj95HSLzUVt83IWs1Xr7ei2DsCEs',
 'object': 'chat.completion',
 'created': 1694945903,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 18, 'completion_tokens': 9, 'total_tokens': 27}}

In [6]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message

{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}

In [7]:
creator_name = "chajasc"

system_instruction = f"""
You are Weather bot created by {creator_name}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {creator_name}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

In [8]:
messages = [
    {"role": "system", "content": system_instruction}
]

In [9]:
functions = [
    {
        "name": "get_current_weather", # name of your function
        "description": "Get the current weather", # description of your function
        "parameters": {
            "type": "object",
            "properties": { # 1st argument of your function
                "location": { # argument name
                    "type": "string", # argument type
                    "description": "The city and state, e.g. San Francisco, CA", # description of your function with example
                }
            },
            "required": ["location"], # required argument of your
        },
    }
]

In [10]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv("RAPID_API_KEY")
    try:
        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q": location}
        print(f"\n>>>> got the querystring as: {querystring}")
        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json
    except Exception as e:
        raise e

In [11]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)

In [12]:
chat_response

<Response [200]>

In [13]:
chat_response.json()

{'id': 'chatcmpl-7zjAQI3LXoFfQaJ5YWrpTBYiGtbnn',
 'object': 'chat.completion',
 'created': 1694945986,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi there! How may I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 163, 'completion_tokens': 11, 'total_tokens': 174}}

In [14]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message

{'role': 'assistant', 'content': 'Hi there! How may I assist you today?'}

# Start fresh

In [16]:
print(system_instruction)


You are Weather bot created by chajasc, You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by chajasc! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. 


In [20]:
messages = list()
messages.append({"role": "system", "content": system_instruction})


user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
print(f"\n>>>> complete_chat_response: \n{chat_response.json()}")
assistant_message = chat_response.json()["choices"][0]["message"]
print(f"\n>>>> assistant message: \n{assistant_message}")


>>>> complete_chat_response: 
{'id': 'chatcmpl-7zjCCX6ZQ81Tx6ES4FUr7XyuWz5MT', 'object': 'chat.completion', 'created': 1694946096, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hi! How can I assist you today?'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 163, 'completion_tokens': 10, 'total_tokens': 173}}

>>>> assistant message: 
{'role': 'assistant', 'content': 'Hi! How can I assist you today?'}


In [21]:
messages.append(assistant_message) # to give context
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by chajasc, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by chajasc! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi there!'},
 {'role': 'assistant', 'content': 'Hi! How can I assist you today?'}]

In [23]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'}},
   'required': ['location']}}]

In [24]:
user_message = "I want to know the weather at Hong Kong"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
print(f"\n>>>> complete_chat_response: \n{chat_response.json()}\n")
assistant_message = chat_response.json()["choices"][0]["message"]
print(f"\n>>>> assistant message: \n{assistant_message}")


>>>> complete_chat_response: 
{'id': 'chatcmpl-7zjEf4KfyXrfdYTe9Uyni6fmNYSXo', 'object': 'chat.completion', 'created': 1694946249, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Hong Kong"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 202, 'completion_tokens': 17, 'total_tokens': 219}}


>>>> assistant message: 
{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Hong Kong"\n}'}}


In [25]:
assistant_message.get("function_call")

{'name': 'get_current_weather', 'arguments': '{\n  "location": "Hong Kong"\n}'}

In [26]:
assistant_message.get("function_call").get("name") # string


'get_current_weather'

In [27]:
assistant_message.get("function_call").get("arguments") # string


'{\n  "location": "Hong Kong"\n}'

In [28]:
import json

def execute_function_call(assistant_message):
    if assistant_message.get("function_call").get("name") == "get_current_weather":
        location = json.loads(assistant_message.get("function_call").get("arguments") )["location"]
        results = get_current_weather(location)
    else:
        results = f"Error: function {assistant_message['function_call']['name']} does not exist"

    return results

In [29]:
results = execute_function_call(assistant_message)
content = json.dumps(results)
content


>>>> got the querystring as: {'q': 'Hong Kong'}

>>>> got the RAPID API response as: 
{'location': {'name': 'Hong Kong', 'region': '', 'country': 'Hong Kong', 'lat': 22.28, 'lon': 114.15, 'tz_id': 'Asia/Hong_Kong', 'localtime_epoch': 1694946248, 'localtime': '2023-09-17 18:24'}, 'current': {'last_updated_epoch': 1694945700, 'last_updated': '2023-09-17 18:15', 'temp_c': 27.6, 'temp_f': 81.7, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 9.8, 'wind_kph': 15.8, 'wind_degree': 109, 'wind_dir': 'ESE', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 84, 'cloud': 26, 'feelslike_c': 31.9, 'feelslike_f': 89.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 13.9, 'gust_kph': 22.3}}


'{"location": {"name": "Hong Kong", "region": "", "country": "Hong Kong", "lat": 22.28, "lon": 114.15, "tz_id": "Asia/Hong_Kong", "localtime_epoch": 1694946248, "localtime": "2023-09-17 18:24"}, "current": {"last_updated_epoch": 1694945700, "last_updated": "2023-09-17 18:15", "temp_c": 27.6, "temp_f": 81.7, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 9.8, "wind_kph": 15.8, "wind_degree": 109, "wind_dir": "ESE", "pressure_mb": 1009.0, "pressure_in": 29.8, "precip_mm": 0.0, "precip_in": 0.0, "humidity": 84, "cloud": 26, "feelslike_c": 31.9, "feelslike_f": 89.4, "vis_km": 10.0, "vis_miles": 6.0, "uv": 7.0, "gust_mph": 13.9, "gust_kph": 22.3}}'

In [30]:
def get_natural_response(content):
    convert_prompt = f"convert this results from weather api to a natural english sentence: {content}"
    messages.append({"role": "user", "content": convert_prompt})
    convert_prompt_response = chat_completion_request(messages=messages)
    print(f"\n>>>> recieved message: {convert_prompt_response.json()}")
    new_assistant_message = convert_prompt_response.json()["choices"][0]["message"]
    messages.append(new_assistant_message)
    content = new_assistant_message["content"]
    print(f"\n>>>> natural response: \n{content}")
    return content


content = get_natural_response(content)



>>>> recieved message: {'id': 'chatcmpl-7zjFvsFZ9xs0XuF2IumNSR5bbdxeI', 'object': 'chat.completion', 'created': 1694946327, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The current weather in Hong Kong is partly cloudy with a temperature of 27.6°C (81.7°F). It feels like 31.9°C (89.4°F) and the wind is coming from the east-southeast at a speed of 15.8 kph (9.8 mph). The air pressure is at 1009.0 mb (29.8 in) and there is no precipitation. The humidity levels are at 84% and visibility is good at 10.0 km (6.0 miles). The UV index is 7.0.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 493, 'completion_tokens': 121, 'total_tokens': 614}}

>>>> natural response: 
The current weather in Hong Kong is partly cloudy with a temperature of 27.6°C (81.7°F). It feels like 31.9°C (89.4°F) and the wind is coming from the east-southeast at a speed of 15.8 kph (9.8 mph). The air pressure is at 1009.0 mb (29.8 in) and there is no precipi

In [31]:
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by chajasc, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by chajasc! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi there!'},
 {'role': 'assistant', 'content': 'Hi! How can I assist you today?'},
 {'role': 'user', 'content': 'I want to know the weather at Hong Kong'},
 {'role': 'user', 'content': 'I want to know the weather at Hong Kong'},
 {'role': 'user',
  'content': 'convert this results from weather api to a natural english sentence: {"location": {"name": "Hong Kong", "region": "", "country": "Hong Kong", "lat": 22.28, "lon": 114.15, "tz_id": 